# Comparing channels

## Nucleotides
- R: sequential
- G: complementary
- B: not match at all

## Proteins
- R: sequential by BLOSUM62
- G: sequential match
- B: sequential by Sneath similarity

In [1]:
import ppscore
import pandas
import os
import numpy
from scipy.stats import spearmanr
from IPython.core.display import display, HTML
from ete3 import Tree
from ete3.parser.newick import NewickError

/tmp/ipykernel_641701/1860748529.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
tree_path = "../../data/trees"
channels = ("P/full", "P/gray_r", "P/gray_g", "P/gray_b", "N/full", "N/gray_r", "N/gray_g", "N/gray_b")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin", "indelible"]

In [3]:
pandas.set_option('display.max_rows', None)
def do_euclidean(control, compare):
    return round(numpy.sqrt(numpy.sum((control.values - compare.values)**2)), 4)

def do_spearman(control, compare):
    return round(spearmanr(control.values.flatten(), compare.values.flatten())[0], 4)

def do_ppscore(control, compare):
    return round(ppscore.score(
                    pandas.DataFrame({"x": control.values.flatten(), "y": compare.values.flatten()}), "x", "y")["ppscore"], 4)

def do_rf(control, compare):
    return round(control.compare(compare, unrooted=True)["norm_rf"], 4)

def do_branch_score(control, compare):
    return 1.0 - round(control.compare(compare, unrooted=True)["source_edges_in_ref"], 4)


def read_and_compare(dataset, channel, score_func, control_name):
    result_dict = dict(
        algorithm=[],
        score=[],
        channel=[],
        dataset=[]
    )
    path = f"{tree_path}/{channel}/{dataset}"
    ends = control_name.split(".")[-1]
    if ends == "nw":
        control = Tree(f"{path}/{control_name}", format=1)
    else:
        control = pandas.read_csv(f"{path}/{control_name}", index_col=0)
    for file in os.listdir(path):
        if file.endswith(ends):
            basename = ".".join(file.split(".")[0:-1])
            try:
                if ends == "nw":
                    compare = Tree(f"{path}/{file}", format=1)
                else:
                    compare = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict["algorithm"] += [basename]
                result_dict["score"] += [score_func(control, compare)]
                result_dict["channel"] += [channel]
                result_dict["dataset"] += [dataset]
            except (FileNotFoundError, NewickError) as e:
                print(f"Error with file {path}/{file}")
    return result_dict

def pprintdf(score_func, control_name="Control with Clustal Omega.csv"):
    result_dict = dict(
        algorithm=[],
        score=[],
        channel=[],
        dataset=[]
    )
    for channel in channels:
        for dataset in datasets:
            d = read_and_compare(dataset, channel, score_func, control_name)
            result_dict["algorithm"] += d["algorithm"]
            result_dict["score"] += d["score"]        
            result_dict["channel"] += d["channel"]        
            result_dict["dataset"] += d["dataset"]
    df = pandas.DataFrame(result_dict)
    table = df.pivot(index=["algorithm", "channel"], columns="dataset", values="score")
    return table

## Euclidean Distance from distance matrices

In [4]:
pprintdf(do_euclidean)

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
                                                   P/gray_b     0.0000   
                                                   P/gray_g     0.0000   
                                                   P/gray_r     0.0000   
Deep Search with Annoy                             N/full       6.1431   
                                                   N/gray_b     5.6131   
                                                   N/gray_g     5.3133   
                                                   N/gray_r     5.3211   
Global with Needleman-Wunsch                       N/full      10.4094   
                                                   N/gray_b    10.4094   
                                                   N/gray_g    10.4094   
                                                   N/gray_r    10.4094   
                                                   P/full       5.4879   
                                                   P/gray_b     5.4879   
                                                   P/gray_g     5.4879   
                                                   P/gray_r     5.4879   
Greedy Sliced Structural Similarity Index Measure  N/full       8.8947   
                                                   N/gray_b     8.9739   
                                                   N/gray_g     8.8143   
                                                   N/gray_r     8.7990   
                                                   P/full       1.4587   
                                                   P/gray_b     2.5123   
                                                   P/gray_g     3.6477   
                                                   P/gray_r     1.5784   
Local with Smith–Waterman                          N/full       8.7200   
                                                   N/gray_b     8.7200   
                                                   N/gray_g     8.7200   
                                                   N/gray_r     8.7200   
                                                   P/full       4.3137   
                                                   P/gray_b     4.3137   
                                                   P/gray_g     4.3137   
                                                   P/gray_r     4.3137   
Resized MultiScale Structural Similarity Index ... N/full       7.4449   
                                                   N/gray_b     7.7001   
                                                   N/gray_g     7.3678   
                                                   N/gray_r     7.2788   
                                                   P/full       4.4038   
                                                   P/gray_b     1.4780   
                                                   P/gray_g     3.9205   
                                                   P/gray_r     8.2075   
Resized Structural Similarity Index Measure        N/full       8.9398   
                                                   N/gray_b     9.0943   
                                                   N/gray_g     8.8752   
                                                   N/gray_r     8.8542   
                                                   P/full       1.4590   
                                                   P/gray_b     2.7293   
                                                   P/gray_g     3.6508   
                                                   P/gray_r     1.6211   
Univ

## Linear Correlation from distance matrices

In [5]:
pprintdf(do_spearman)

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       1.0000   
                                                   N/gray_b     1.0000   
                                                   N/gray_g     1.0000   
                                                   N/gray_r     1.0000   
                                                   P/full       1.0000   
                                                   P/gray_b     1.0000   
                                                   P/gray_g     1.0000   
                                                   P/gray_r     1.0000   
Deep Search with Annoy                             N/full       0.0771   
                                                   N/gray_b     0.1702   
                                                   N/gray_g     0.0638   
                                                   N/gray_r     0.0498   
Global with Needleman-Wunsch                       N/full       0.2541   
                                                   N/gray_b     0.2541   
                                                   N/gray_g     0.2541   
                                                   N/gray_r     0.2541   
                                                   P/full       0.3039   
                                                   P/gray_b     0.3039   
                                                   P/gray_g     0.3039   
                                                   P/gray_r     0.3039   
Greedy Sliced Structural Similarity Index Measure  N/full       0.2356   
                                                   N/gray_b     0.2032   
                                                   N/gray_g     0.2812   
                                                   N/gray_r     0.2865   
                                                   P/full       0.1563   
                                                   P/gray_b     0.2782   
                                                   P/gray_g     0.1614   
                                                   P/gray_r     0.2321   
Local with Smith–Waterman                          N/full       0.3428   
                                                   N/gray_b     0.3428   
                                                   N/gray_g     0.3428   
                                                   N/gray_r     0.3428   
                                                   P/full       0.2445   
                                                   P/gray_b     0.2445   
                                                   P/gray_g     0.2445   
                                                   P/gray_r     0.2445   
Resized MultiScale Structural Similarity Index ... N/full       0.1944   
                                                   N/gray_b     0.2302   
                                                   N/gray_g     0.1641   
                                                   N/gray_r     0.2447   
                                                   P/full       0.1988   
                                                   P/gray_b     0.2492   
                                                   P/gray_g     0.1862   
                                                   P/gray_r     0.2028   
Resized Structural Similarity Index Measure        N/full       0.2625   
                                                   N/gray_b     0.2118   
                                                   N/gray_g     0.2422   
                                                   N/gray_r     0.2960   
                                                   P/full       0.1518   
                                                   P/gray_b     0.2503   
                                                   P/gray_g     0.1560   
                                                   P/gray_r     0.2277   
Univ

## Predictive Power Score from distance matrices

In [6]:
pprintdf(do_ppscore)

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.9998   
                                                   N/gray_b     0.9998   
                                                   N/gray_g     0.9998   
                                                   N/gray_r     0.9998   
                                                   P/full       0.9992   
                                                   P/gray_b     0.9992   
                                                   P/gray_g     0.9992   
                                                   P/gray_r     0.9992   
Deep Search with Annoy                             N/full       0.6019   
                                                   N/gray_b     0.5553   
                                                   N/gray_g     0.5866   
                                                   N/gray_r     0.5755   
Global with Needleman-Wunsch                       N/full       0.9039   
                                                   N/gray_b     0.9039   
                                                   N/gray_g     0.9039   
                                                   N/gray_r     0.9039   
                                                   P/full       0.8767   
                                                   P/gray_b     0.8767   
                                                   P/gray_g     0.8767   
                                                   P/gray_r     0.8767   
Greedy Sliced Structural Similarity Index Measure  N/full       0.9803   
                                                   N/gray_b     0.9569   
                                                   N/gray_g     0.9843   
                                                   N/gray_r     0.9812   
                                                   P/full       0.9330   
                                                   P/gray_b     0.9525   
                                                   P/gray_g     0.8368   
                                                   P/gray_r     0.9284   
Local with Smith–Waterman                          N/full       0.9779   
                                                   N/gray_b     0.9779   
                                                   N/gray_g     0.9779   
                                                   N/gray_r     0.9779   
                                                   P/full       0.9643   
                                                   P/gray_b     0.9643   
                                                   P/gray_g     0.9643   
                                                   P/gray_r     0.9643   
Resized MultiScale Structural Similarity Index ... N/full       0.9631   
                                                   N/gray_b     0.9369   
                                                   N/gray_g     0.9581   
                                                   N/gray_r     0.9576   
                                                   P/full       0.8480   
                                                   P/gray_b     0.7416   
                                                   P/gray_g     0.8492   
                                                   P/gray_r     0.7881   
Resized Structural Similarity Index Measure        N/full       0.9914   
                                                   N/gray_b     0.9870   
                                                   N/gray_g     0.9894   
                                                   N/gray_r     0.9896   
                                                   P/full       0.9346   
                                                   P/gray_b     0.9294   
                                                   P/gray_g     0.8422   
                                                   P/gray_r     0.9256   
Univ

## Robinson-foulds distance from generated Trees

In [7]:
pprintdf(do_rf, control_name="Control with Clustal Omega.nw")

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
                                                   P/gray_b     0.0000   
                                                   P/gray_g     0.0000   
                                                   P/gray_r     0.0000   
Deep Search with Annoy                             N/full       0.9730   
                                                   N/gray_b     1.0000   
                                                   N/gray_g     1.0000   
                                                   N/gray_r     1.0000   
Global with Needleman-Wunsch                       N/full       0.1622   
                                                   N/gray_b     0.1622   
                                                   N/gray_g     0.1622   
                                                   N/gray_r     0.1622   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1892   
Greedy Sliced Structural Similarity Index Measure  N/full       0.1892   
                                                   N/gray_b     0.1892   
                                                   N/gray_g     0.1892   
                                                   N/gray_r     0.1892   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1622   
                                                   P/gray_g     0.1622   
                                                   P/gray_r     0.1892   
Local with Smith–Waterman                          N/full       0.1622   
                                                   N/gray_b     0.1622   
                                                   N/gray_g     0.1622   
                                                   N/gray_r     0.1622   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1892   
Resized MultiScale Structural Similarity Index ... N/full       0.1892   
                                                   N/gray_b     0.1351   
                                                   N/gray_g     0.1892   
                                                   N/gray_r     0.1892   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1622   
                                                   P/gray_r     0.1622   
Resized Structural Similarity Index Measure        N/full       0.1892   
                                                   N/gray_b     0.1892   
                                                   N/gray_g     0.1892   
                                                   N/gray_r     0.1892   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1622   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1892   
Univ

In [8]:
pprintdf(do_rf, control_name="Global with Needleman-Wunsch.nw")

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.1622   
                                                   N/gray_b     0.1622   
                                                   N/gray_g     0.1622   
                                                   N/gray_r     0.1622   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1892   
Deep Search with Annoy                             N/full       0.9730   
                                                   N/gray_b     1.0000   
                                                   N/gray_g     1.0000   
                                                   N/gray_r     1.0000   
Global with Needleman-Wunsch                       N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
                                                   P/gray_b     0.0000   
                                                   P/gray_g     0.0000   
                                                   P/gray_r     0.0000   
Greedy Sliced Structural Similarity Index Measure  N/full       0.1892   
                                                   N/gray_b     0.1892   
                                                   N/gray_g     0.1892   
                                                   N/gray_r     0.1892   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1892   
Local with Smith–Waterman                          N/full       0.1622   
                                                   N/gray_b     0.1622   
                                                   N/gray_g     0.1622   
                                                   N/gray_r     0.1622   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1892   
Resized MultiScale Structural Similarity Index ... N/full       0.1892   
                                                   N/gray_b     0.1622   
                                                   N/gray_g     0.1892   
                                                   N/gray_r     0.1892   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1622   
                                                   P/gray_r     0.1892   
Resized Structural Similarity Index Measure        N/full       0.1892   
                                                   N/gray_b     0.1892   
                                                   N/gray_g     0.1892   
                                                   N/gray_r     0.1622   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1892   
Univ

In [9]:
pprintdf(do_rf, control_name="Local with Smith–Waterman.nw")

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.1622   
                                                   N/gray_b     0.1622   
                                                   N/gray_g     0.1622   
                                                   N/gray_r     0.1622   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1892   
Deep Search with Annoy                             N/full       0.9730   
                                                   N/gray_b     1.0000   
                                                   N/gray_g     1.0000   
                                                   N/gray_r     1.0000   
Global with Needleman-Wunsch                       N/full       0.1622   
                                                   N/gray_b     0.1622   
                                                   N/gray_g     0.1622   
                                                   N/gray_r     0.1622   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1892   
Greedy Sliced Structural Similarity Index Measure  N/full       0.1892   
                                                   N/gray_b     0.1892   
                                                   N/gray_g     0.1892   
                                                   N/gray_r     0.1892   
                                                   P/full       0.1622   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1622   
                                                   P/gray_r     0.1622   
Local with Smith–Waterman                          N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
                                                   P/gray_b     0.0000   
                                                   P/gray_g     0.0000   
                                                   P/gray_r     0.0000   
Resized MultiScale Structural Similarity Index ... N/full       0.1892   
                                                   N/gray_b     0.1892   
                                                   N/gray_g     0.1892   
                                                   N/gray_r     0.1892   
                                                   P/full       0.1892   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1892   
                                                   P/gray_r     0.1622   
Resized Structural Similarity Index Measure        N/full       0.1892   
                                                   N/gray_b     0.1892   
                                                   N/gray_g     0.1892   
                                                   N/gray_r     0.1892   
                                                   P/full       0.1622   
                                                   P/gray_b     0.1892   
                                                   P/gray_g     0.1622   
                                                   P/gray_r     0.1892   
Univ

## Compatibility branch score from generated Trees

In [10]:
pprintdf(do_branch_score, control_name="Control with Clustal Omega.nw")

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
                                                   P/gray_b     0.0000   
                                                   P/gray_g     0.0000   
                                                   P/gray_r     0.0000   
Deep Search with Annoy                             N/full       0.4737   
                                                   N/gray_b     0.4868   
                                                   N/gray_g     0.4868   
                                                   N/gray_r     0.4868   
Global with Needleman-Wunsch                       N/full       0.0789   
                                                   N/gray_b     0.0789   
                                                   N/gray_g     0.0789   
                                                   N/gray_r     0.0789   
                                                   P/full       0.0921   
                                                   P/gray_b     0.0921   
                                                   P/gray_g     0.0921   
                                                   P/gray_r     0.0921   
Greedy Sliced Structural Similarity Index Measure  N/full       0.0921   
                                                   N/gray_b     0.0921   
                                                   N/gray_g     0.0921   
                                                   N/gray_r     0.0921   
                                                   P/full       0.0921   
                                                   P/gray_b     0.0789   
                                                   P/gray_g     0.0789   
                                                   P/gray_r     0.0921   
Local with Smith–Waterman                          N/full       0.0789   
                                                   N/gray_b     0.0789   
                                                   N/gray_g     0.0789   
                                                   N/gray_r     0.0789   
                                                   P/full       0.0921   
                                                   P/gray_b     0.0921   
                                                   P/gray_g     0.0921   
                                                   P/gray_r     0.0921   
Resized MultiScale Structural Similarity Index ... N/full       0.0921   
                                                   N/gray_b     0.0658   
                                                   N/gray_g     0.0921   
                                                   N/gray_r     0.0921   
                                                   P/full       0.0921   
                                                   P/gray_b     0.0921   
                                                   P/gray_g     0.0789   
                                                   P/gray_r     0.0789   
Resized Structural Similarity Index Measure        N/full       0.0921   
                                                   N/gray_b     0.0921   
                                                   N/gray_g     0.0921   
                                                   N/gray_r     0.0921   
                                                   P/full       0.0921   
                                                   P/gray_b     0.0789   
                                                   P/gray_g     0.0921   
                                                   P/gray_r     0.0921   
Univ